In [ ]:
os.makedirs("uniteler_txt", exist_ok=True)
os.makedirs("unite_haritalari", exist_ok=True)

print("Lütfen .txt dosyalarını 'uniteler_txt' klasörüne yükle")

Lütfen .txt dosyalarını 'uniteler_txt' klasörüne yükle


In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3")

SYSTEM_PRTOMPY = SYSTEM_PROMPT = """
Sen bir AÖF (Açıköğretim Fakültesi) Akademik Veri Analistisin.
Görevin, sana verilen ünite metnini analiz ederek bir "Bilgi Navigasyon Haritası" oluşturmaktır.

Lütfen aşağıdaki yapıyı kullanarak metni işle ve SADECE saf JSON formatında çıktı ver (başka açıklama ekleme):

{
  "unite_kuyne": "Ünite No ve Adı",
  "ana_basliklar": ["Başlık 1", "Başlık 2"],
  "kritik_kavramlar": [
    {"terim": "Kavram Adı", "tanim": "Metne sadık kısa tanım"}
  ],
  "aktorler_ve_ekoller": [
    {"isim": "Kurum/Ekol/Kişi Adı", "gorus": "Temel görüşü"}
  ],
  "sinav_odakli_tahmin": "Hangi konudan soru gelme ihtimali yüksek?"
}

Kurallar: Sadece metindeki bilgiyi kullan, dili tamamen Türkçe olsun.
"""

def haritalari_baslat():
    giris_yolu = "/content/drive/MyDrive/sosyalpolitikaOzetTXT"
    cikis_yolu = "unite_haritalari"

    dosyalar = [f for f in os.listdir(giris_yolu) if f.endswith(".txt")]

    if not dosyalar:
        print("Hata: 'uniteler_txt' içinde dosya bulunamadı!")
        return

    for dosya_adi in dosyalar:
        print(f"İşleniyor: {dosya_adi}...")
        with open(os.path.join(giris_yolu, dosya_adi), "r", encoding="utf-8") as f:
            metin = f.read()

        full_prompt = f"{SYSTEM_PROMPT}\n\nMetin:\n{metin}"
        response = llm.invoke(full_prompt)

        with open(os.path.join(cikis_yolu, dosya_adi.replace(".txt", ".json")), "w", encoding="utf-8") as f:
            f.write(response)
        print(f"Tamamlandı: {dosya_adi}")

# Çalıştır
haritalari_baslat()

İşleniyor: sosyalpolitikaozet16.txt...
Tamamlandı: sosyalpolitikaozet16.txt
İşleniyor: sosyalpolitikaozet15.txt...
Tamamlandı: sosyalpolitikaozet15.txt
İşleniyor: sosyalpolitikaozet14.txt...
Tamamlandı: sosyalpolitikaozet14.txt
İşleniyor: sosyalpolitikaozet13.txt...
Tamamlandı: sosyalpolitikaozet13.txt
İşleniyor: sosyalpolitikaozet12.txt...
Tamamlandı: sosyalpolitikaozet12.txt
İşleniyor: sosyalpolitikaozet11.txt...
Tamamlandı: sosyalpolitikaozet11.txt
İşleniyor: sosyalpolitikaozet10.txt...
Tamamlandı: sosyalpolitikaozet10.txt
İşleniyor: sosyalpolitikaozet9.txt...
Tamamlandı: sosyalpolitikaozet9.txt


In [ ]:
import re
import json

def sinav_formatla(ham_metin, cevap_anahtari_str):
    # 1. Gereksiz kurum mesajlarını ve sayfa numaralarını temizle
    temiz_metin = re.sub(r"Açıköğretim Sistemi ile ilgili.*?yasaktır\.", "", ham_metin, flags=re.DOTALL)
    temiz_metin = re.sub(r"Anadolu Üniversitesi.*?sayılır\.", "", temiz_metin, flags=re.DOTALL)
    temiz_metin = re.sub(r"\d{4} - \d{4} Öğretim Yılı|Yaz Okulu Sınavı|SOSYAL POLİTİKA A|2021 YAZ OKULU - A|\d+", "", temiz_metin)

    # 2. Cevap anahtarını sözlüğe çevir (Örn: {1: 'A', 2: 'E'...})
    cevaplar = cevap_anahtari_str.split()
    cevap_anahtari = {i+1: cevaplar[i] for i in range(len(cevaplar))}

    # 3. Soruları ayıkla (Soru metni ve Şıkları yakalayan RegEx)
    # Bu desen: Soru metnini alır, A'dan E'ye kadar olan şıkları gruplar.
    soru_deseni = re.compile(r"(.*?)\s+A\)\s+(.*?)\s+B\)\s+(.*?)\s+C\)\s+(.*?)\s+D\)\s+(.*?)\s+E\)\s+(.*?)(?=\s+|$)", re.DOTALL)

    bulunan_sorular = soru_deseni.findall(ham_metin)

    altin_liste = []
    for i, q in enumerate(bulunan_sorular):
        soru_no = i + 1
        obj = {
            "SORU_ID": f"2021_YAZ_{soru_no}",
            "SORU": q[0].strip().replace("\n", " "),
            "SIKLAR": {
                "A": q[1].strip(),
                "B": q[2].strip(),
                "C": q[3].strip(),
                "D": q[4].strip(),
                "E": q[5].strip()
            },
            "CEVAP": cevap_anahtari.get(soru_no, "Bilinmiyor"),
            "UNITE_NO": "Bilinmiyor (Analiz Gerekli)",
            "ANAHTAR_KAVRAMLAR": []
        }
        altin_liste.append(obj)

    return altin_liste

# --- KULLANIM ---
ham_veri = """
1.1936 yılında yazdığı “İstihdam, Faiz ve
Paranın Genel Teorisi” adlı eserinde, klasik
iktisat anlayışına önemli eleştiriler getirerek
ve liberal çizgiden ödünler vermiş olan
iktisatçı kimdir?
A)
Keynes
B)
Friedman
C)
Fisher
D)
Hayek
E)
Mill

2.Sanayi Devrimi’nin ortaya çıkardığı kötü
çalışma koşullarına karşı işçileri ve emeği
sermayeye karşı korumak ve bu yolla
toplumdaki sınıf çatışmalarını önleyerek
toplumun ve devletin varlığını sürdürmesini
sağlamaya yönelik uygulamalara kavramsal
açıdan ne ad verilir?
A)
Sosyal yardım
B)
Geniş anlamda sosyal politika
C)
Sosyal sigorta
D)
Sosyal güvenlik
E)

3.Dar anlamda sosyal politika
Sanayi Devrimi'nin yaşandığı yıllarda, emek
sahiplerinin elde ettiği ücret gelirinin temel
ihtiyaçlar düzeyini karşılamakta yeterli
olmadığı en düşük ücret düzeyine ne ad
verilir?
A)
Sosyal ücret
B)
Sefalet ücreti
C)
Kök Ücret
D)
Temel ücret
E)
Asgari ücret

4.Aşağıdakilerden hangisi, liberalizmin
ekonomik ve sosyal etkilerine karşı bir fikir
olarak ortaya çıkan sosyalizm
temsilcilerinden biri değildir?
A)
Charles Fourier
B)
Saint Simon
C)
Robert Owen
D)
Robert Thomas Malthus
E)
Karl Marks
I. Kamu müdahalesi
II. Üçüncü yol
III. Yeşiller
IV. Feminist hareketler
V. Avrupa Birliği
5.Yukarıdakilerden hangileri uluslararası
sosyal politika araçlarındandır?
A)
I ve II
B)
I ve IV
C)
I, II ve III
D)
II, III ve IV
E)
II, III, IV ve V

6.Beslenme, çocuk ölüm oranları, eğitimde
geçirilen süre, çocukların kayıt altına
alınması, yemek pişirme olanakları, tuvalet,
su, elektrik, zemin döşemesi ve sahip olunan
mal varlıkları göstergelerinden oluşan
endeks aşağıdakilerden hangisidir?

A)
Lorenz Eğrisi
B)
İnsanî Yoksulluk Endeksi
C)
Kafa Eğrisi
D)
Gini Katsayısı
E)

7.Çok Boyutlu Yoksulluk Endeksi
Aşağıdakilerden hangisi çalışma hakkı
çerçevesinde düzenlenen sosyal politika
tedbirlerinden biri değildir?
Çalışma hayatında belirli bir gelir
güvencesinin sağlanması
A)
Ücretlerin fiyat yükselişleri karşısında reel
satın alma gücünün korunması
B)
Ücretlilere örgütlenme hakkının
kazandırılması
C)
Aktif ve pasif istihdam politikaları aracılığıyla
işsizlik karşısında korunulması
D)
Çalışma hayatında iş güvencesi ve
koruyucu standartların sağlanması
E)

8.Çalışanın; verimliliği, eğitim düzeyi, kıdemi
gibi unsurları dikkate almaksızın
kararlaştırılan ödemenin belirli bir süre
çalışılması sonunda hak edildiği ücret
sistemi aşağıdakilerden hangisidir?
A)
Miktar esasına dayalı ücret sistemi
B)
Götürü ücret sistemi
C)
Zaman esasına dayalı ücret sistemi
D)
Verimlilik esasına dayalı ücret sistemi
E)
9.Performansa dayalı ücret sistemi
Aşağıdakilerden hangisi pasif istihdam
politikalarından biridir?
A)
Doğrudan kamu istihdamı
B)
Mesleki eğitim programları
C)
İş paylaşımı
D)
Ücret ve istihdam sübvansiyonları
Girişimciliğin desteklenmesine yönelik
programlar
E)

10.Alınması gereken günlük kalori miktarı ya da
yapılan tüketim harcamaları gibi tespit
edilebilir ve doğruluğu kanıtlanabilir bir
standardın ya da standartlar setinin
aşağısında kalma nedeniyle ortaya çıkan
yoksulluğa ne ad verilir?
A)
Göreli yoksulluk
B)
Sübjektif yoksulluk
C)
Mutlak yoksulluk
D)
Objektif yoksulluk
E)
İnsani yoksulluk

11.Üretim sürecine katılan üretim faktörleri
tarafından yaratılan gelirin serbest piyasa
ekonomisine hiçbir müdahale olmaksızın
dağılımına ne ad verilir?
A)
Müdahale öncesi gelir dağılımı
B)
İkincil gelir dağılımı
C)
Birincil gelir dağılımı
D)
Fonksiyonel gelir dağılımı
E)
Düzenli gelir dağılımı

12.Avrupa Birliği sosyal dışlanma sorununa ilk
olarak aşağıdaki belgelerden hangisinde açık
bir biçimde yer vermiştir?
A)
Sosyal Dışlanma Protokolü
B)
AB Anayasası
C)
Roma Antlaşması
D)
Sosyal Politika Antlaşması
E)
Amsterdam Antlaşması

13.Meydana getiren faktörler (sebepler)
açısından hastalık, ne tür bir tehlike (risk)
olarak kabul edilir?
A)
Sosyo-ekonomik
B)
Fizyolojik
C)
İnsanların sebep olduğu
D)
Tabii afetlerden kaynaklanan
E)
Ekonomik

14.Ayrımcılık yasağına ilişkin Birleşmiş
Milletler’e ait ilk düzenleme aşağıdakilerden
hangisidir?
A)
Birleşmiş Milletler Anlaşması
B)
İnsan Hakları Evrensel Bildirgesi
Ekonomik Sosyal ve Kültürel Haklar
Sözleşmesi
C)
Her Türlü Irk Ayrımcılığının Ortadan
Kaldırılmasına İlişkin Uluslararası Sözleşme
D)
E)

15.Medeni ve Siyasi Haklar Sözleşmesi
Sosyal dışlanma kavramı ilk olarak hangi
ülkede kullanılmıştır?
A)
Fransa
B)
İngiltere
C)
Almanya
D)
Rusya
E)
ABD

16.Sosyal sigortaların en son kurulan sigorta
kolu aşağıdakilerden hangisidir?
A)
Malullük
B)
Yaşlılık
C)
Hastalık
D)
İşsizlik
E)
Ölüm

17.Aşağıdakilerden hangisi sosyal sigortaların
özelliklerinden biri değildir?
Ödenen primlerle sağlanan haklar arasında
bire bir ilişki olması
A)
B)
Devletin sosyal sigortaların garantörü olması
C)
Yönetimine sosyal tarafların da katılması
D)
Sosyal sigortalara katılmanın zorunlu olması
Devlet tarafından kurulan bir sigorta
programı olması
E)

18.Birleşmiş Milletler, kendi vatandaşı olduğu
ülkeden başka bir ülkede bir yıl ve daha fazla
süre kalan kişilere ------- adı verir.
Yukarıdaki cümlede boş bırakılan yeri
aşağıdakilerden hangisi doğru olarak
tamamlar?
A)
sığınmacı
B)
göçmen
C)
ilticacı
D)
kaçak
E)
mülteci

19.Düşük ücret, uzun çalışma süreleri ve ağır
çalışma koşulları altında çalıştırılmalarının
toplumsal tepkilere neden olması üzerine
aşağıdaki ülkelerin hangisinden başlamak
üzere kadınların çalışma yaşamında özel
olarak korunmaları ile ilgili hukuki
düzenlemeler yapılmaya başlanmıştır?
A)
Almanya
B)
İsveç
C)
İngiltere
D)
Küba
E)
ABD

20.Yaşlılığın tanımlanmasında göreli daha fazla
kullanılan kavram aşağıdakilerden
hangisidir?
A)
İleri yaşlılık
B)
Biyolojik yaşlılık
C)
Yasal yaşlılık
D)
Orta yaşlılık
E)

"""
cevap_anahtari = """A E B D E E B C C D C D B A A D A B C E"""

sonuc = sinav_formatla(ham_veri, cevap_anahtari)

# JSON olarak kaydet
with open("altin_sorular.json", "w", encoding="utf-8") as f:
    json.dump(sonuc, f, ensure_ascii=False, indent=2)

print(f"{len(sonuc)} soru başarıyla formatlandı!")

18 soru başarıyla formatlandı!


In [ ]:
import re
import json

def altin_format_olustur(ham_metin, cevap_anahtari_str):
    # 1. Metni Temizle (Kurum mesajlarını ve sayfa numaralarını sil)
    temiz_metin = re.sub(r"Açıköğretim Sistemi ile ilgili.*?yasaktır\.", "", ham_metin, flags=re.DOTALL)
    temiz_metin = re.sub(r"Anadolu Üniversitesi.*?sayılır\.", "", temiz_metin, flags=re.DOTALL)
    # Sayfa numaraları ve sınav başlıklarını temizle
    temiz_metin = re.sub(r"\d{4} - \d{4} Öğretim Yılı|Yaz Okulu Sınavı|SOSYAL POLİTİKA A|2021 YAZ OKULU - A", "", temiz_metin)

    # 2. Cevap Anahtarını Hazırla
    cevap_listesi = cevap_anahtari_str.split()

    # 3. Soruları Ayıkla
    # Soru numarası (örn: 1.) ile başlayan blokları bulur
    soru_bloklari = re.split(r'\n(?=\d+\.)', temiz_metin)

    altin_liste = []
    soru_sayaci = 0

    for blok in soru_bloklari:
        # Soru numarası ve içeriği ayır
        match = re.match(r'(\d+)\.(.*)', blok.strip(), re.DOTALL)
        if match:
            soru_no_text = match.group(1)
            icerik = match.group(2)

            # Şıkları parçala (A, B, C, D, E)
            parts = re.split(r'\s([A-E])\)\s', icerik)

            if len(parts) > 1:
                soru_metni = parts[0].strip().replace('\n', ' ')
                siklar = {}
                for i in range(1, len(parts), 2):
                    if i+1 < len(parts):
                        sik_harfi = parts[i]
                        # Şık metnindeki gereksiz satır sonlarını temizle
                        sik_metni = parts[i+1].split('\n')[0].strip()
                        siklar[sik_harfi] = sik_metni

                # Soruya doğru cevabı ata
                soru_cevap = cevap_listesi[soru_sayaci] if soru_sayaci < len(cevap_listesi) else "Bilinmiyor"

                altin_liste.append({
                    "SORU_ID": f"AOF_2021_YAZ_{soru_no_text}",
                    "SORU": soru_metni,
                    "SIKLAR": siklar,
                    "CEVAP": soru_cevap,
                    "METADATA": {
                        "unite_no": None,
                        "kavramlar": []
                    }
                })
                soru_sayaci += 1

    return altin_liste

# --- UYGULAMA ---

ham_veri = """1.1936 yılında yazdığı “İstihdam, Faiz ve
Paranın Genel Teorisi” adlı eserinde, klasik
iktisat anlayışına önemli eleştiriler getirerek
ve liberal çizgiden ödünler vermiş olan
iktisatçı kimdir?
A)
Keynes
B)
Friedman
C)
Fisher
D)
Hayek
E)
Mill

2.Sanayi Devrimi’nin ortaya çıkardığı kötü
çalışma koşullarına karşı işçileri ve emeği
sermayeye karşı korumak ve bu yolla
toplumdaki sınıf çatışmalarını önleyerek
toplumun ve devletin varlığını sürdürmesini
sağlamaya yönelik uygulamalara kavramsal
açıdan ne ad verilir?
A) Sosyal yardım
B) Geniş anlamda sosyal politika
C) Sosyal sigorta
D) Sosyal güvenlik
E) Dar anlamda sosyal politika

3.Sanayi Devrimi'nin yaşandığı yıllarda, emek
sahiplerinin elde ettiği ücret gelirinin temel
ihtiyaçlar düzeyini karşılamakta yeterli
olmadığı en düşük ücret düzeyine ne ad
verilir?
A)
Sosyal ücret
B)
Sefalet ücreti
C)
Kök Ücret
D)
Temel ücret
E)
Asgari ücret

4.Aşağıdakilerden hangisi, liberalizmin
ekonomik ve sosyal etkilerine karşı bir fikir
olarak ortaya çıkan sosyalizm
temsilcilerinden biri değildir?
A)
Charles Fourier
B)
Saint Simon
C)
Robert Owen
D)
Robert Thomas Malthus
E)
Karl Marks
I. Kamu müdahalesi
II. Üçüncü yol
III. Yeşiller
IV. Feminist hareketler
V. Avrupa Birliği
5.Yukarıdakilerden hangileri uluslararası
sosyal politika araçlarındandır?
A)
I ve II
B)
I ve IV
C)
I, II ve III
D)
II, III ve IV
E)
II, III, IV ve V

6.Beslenme, çocuk ölüm oranları, eğitimde
geçirilen süre, çocukların kayıt altına
alınması, yemek pişirme olanakları, tuvalet,
su, elektrik, zemin döşemesi ve sahip olunan
mal varlıkları göstergelerinden oluşan
endeks aşağıdakilerden hangisidir?

A)
Lorenz Eğrisi
B)
İnsanî Yoksulluk Endeksi
C)
Kafa Eğrisi
D)
Gini Katsayısı
E) Çok Boyutlu Yoksulluk Endeksi

7.Aşağıdakilerden hangisi çalışma hakkı
çerçevesinde düzenlenen sosyal politika
tedbirlerinden biri değildir?

A)Çalışma hayatında belirli bir gelir
güvencesinin sağlanması

B)Ücretlerin fiyat yükselişleri karşısında reel
satın alma gücünün korunması

C)Ücretlilere örgütlenme hakkının
kazandırılması

D)Aktif ve pasif istihdam politikaları aracılığıyla
işsizlik karşısında korunulması

E)Çalışma hayatında iş güvencesi ve
koruyucu standartların sağlanması

8.Çalışanın; verimliliği, eğitim düzeyi, kıdemi
gibi unsurları dikkate almaksızın
kararlaştırılan ödemenin belirli bir süre
çalışılması sonunda hak edildiği ücret
sistemi aşağıdakilerden hangisidir?
A)
Miktar esasına dayalı ücret sistemi
B)
Götürü ücret sistemi
C)
Zaman esasına dayalı ücret sistemi
D)
Verimlilik esasına dayalı ücret sistemi
E)
9.Performansa dayalı ücret sistemi
Aşağıdakilerden hangisi pasif istihdam
politikalarından biridir?
A)
Doğrudan kamu istihdamı
B)
Mesleki eğitim programları
C)
İş paylaşımı
D)
Ücret ve istihdam sübvansiyonları
E)
Girişimciliğin desteklenmesine yönelik
programlar

10.Alınması gereken günlük kalori miktarı ya da
yapılan tüketim harcamaları gibi tespit
edilebilir ve doğruluğu kanıtlanabilir bir
standardın ya da standartlar setinin
aşağısında kalma nedeniyle ortaya çıkan
yoksulluğa ne ad verilir?
A)
Göreli yoksulluk
B)
Sübjektif yoksulluk
C)
Mutlak yoksulluk
D)
Objektif yoksulluk
E)
İnsani yoksulluk

11.Üretim sürecine katılan üretim faktörleri
tarafından yaratılan gelirin serbest piyasa
ekonomisine hiçbir müdahale olmaksızın
dağılımına ne ad verilir?
A)
Müdahale öncesi gelir dağılımı
B)
İkincil gelir dağılımı
C)
Birincil gelir dağılımı
D)
Fonksiyonel gelir dağılımı
E)
Düzenli gelir dağılımı

12.Avrupa Birliği sosyal dışlanma sorununa ilk
olarak aşağıdaki belgelerden hangisinde açık
bir biçimde yer vermiştir?
A)
Sosyal Dışlanma Protokolü
B)
AB Anayasası
C)
Roma Antlaşması
D)
Sosyal Politika Antlaşması
E)
Amsterdam Antlaşması

13.Meydana getiren faktörler (sebepler)
açısından hastalık, ne tür bir tehlike (risk)
olarak kabul edilir?
A)
Sosyo-ekonomik
B)
Fizyolojik
C)
İnsanların sebep olduğu
D)
Tabii afetlerden kaynaklanan
E)
Ekonomik

14.Ayrımcılık yasağına ilişkin Birleşmiş
Milletler’e ait ilk düzenleme aşağıdakilerden
hangisidir?
A)
Birleşmiş Milletler Anlaşması
B)
İnsan Hakları Evrensel Bildirgesi
C)
Ekonomik Sosyal ve Kültürel Haklar
Sözleşmesi
D)
Her Türlü Irk Ayrımcılığının Ortadan
E)
Kaldırılmasına İlişkin Uluslararası Sözleşme

15.Medeni ve Siyasi Haklar Sözleşmesi
Sosyal dışlanma kavramı ilk olarak hangi
ülkede kullanılmıştır?
A)
Fransa
B)
İngiltere
C)
Almanya
D)
Rusya
E)
ABD

16.Sosyal sigortaların en son kurulan sigorta
kolu aşağıdakilerden hangisidir?
A)
Malullük
B)
Yaşlılık
C)
Hastalık
D)
İşsizlik
E)
Ölüm

17.Aşağıdakilerden hangisi sosyal sigortaların
özelliklerinden biri değildir?
A) Ödenen primlerle sağlanan haklar arasında
bire bir ilişki olması
B)
Devletin sosyal sigortaların garantörü olması
C)
Yönetimine sosyal tarafların da katılması
D)
Sosyal sigortalara katılmanın zorunlu olması
E)
Devlet tarafından kurulan bir sigorta
programı olması

18.Birleşmiş Milletler, kendi vatandaşı olduğu
ülkeden başka bir ülkede bir yıl ve daha fazla
süre kalan kişilere ------- adı verir.
Yukarıdaki cümlede boş bırakılan yeri
aşağıdakilerden hangisi doğru olarak
tamamlar?
A)
sığınmacı
B)
göçmen
C)
ilticacı
D)
kaçak
E)
mülteci

19.Düşük ücret, uzun çalışma süreleri ve ağır
çalışma koşulları altında çalıştırılmalarının
toplumsal tepkilere neden olması üzerine
aşağıdaki ülkelerin hangisinden başlamak
üzere kadınların çalışma yaşamında özel
olarak korunmaları ile ilgili hukuki
düzenlemeler yapılmaya başlanmıştır?
A)
Almanya
B)
İsveç
C)
İngiltere
D)
Küba
E)
ABD

20.Yaşlılığın tanımlanmasında göreli daha fazla
kullanılan kavram aşağıdakilerden
hangisidir?
A)
İleri yaşlılık
B)
Biyolojik yaşlılık
C)
Yasal yaşlılık
D)
Orta yaşlılık
E)
Kronik Yaşlılık

"""
cevap_anahtari = "A E B D E E B C C D C D B A A D A B C E"

# Fonksiyonu çalıştır
sonuc_listesi = altin_format_olustur(ham_veri, cevap_anahtari)

# JSON olarak kaydet
with open("altin_sorular.json", "w", encoding="utf-8") as f:
    json.dump(sonuc_listesi, f, ensure_ascii=False, indent=2)

print(f"İşlem Tamam: {len(sonuc_listesi)} soru cevaplarıyla birlikte 'altin_sorular.json' dosyasına kaydedildi.")

İşlem Tamam: 19 soru cevaplarıyla birlikte 'altin_sorular.json' dosyasına kaydedildi.


In [ ]:
import json
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3")
def json_temizle(raw_text):
    """LLM yanıtındaki gereksiz metinleri temizleyip sadece JSON kısmını alır."""
    try:
        # { ile başlayan ve } ile biten en geniş bloğu bul
        match = re.search(r'\{.*\}', raw_text, re.DOTALL)
        if match:
            return match.group(0)
        return raw_text
    except:
        return raw_text
# Daha önce oluşturduğun Ünite Haritalarını yükle (Örnek: unite_6.json)
# Sistem bu haritadaki kavramlarla soruyu eşleştirecek
def kavramlari_etiketle(sorular_listesi):

    SYSTEM_PROMPT = """
    Sen bir akademik içerik etiketleyicisin. Sana bir soru vereceğim.
    Bu sorunun temel konusunu, içinde geçen kritik kavramları ve hangi üniteyle ilgili olduğunu tespit et.

    ÇIKTI FORMATI (Sadece JSON):
    {
         "kavramlar": ["Sosyal Dışlanma", "Rene Lenoir", "Fransa"]
    }
    """

    for soru in sorular_listesi:
        print(f"İşleniyor: {soru['SORU_ID']}")

        prompt = f"{SYSTEM_PROMPT}\n\nSoru: {soru['SORU']}\nŞıklar: {soru['SIKLAR']}"

        # LLM'den analiz iste
        analiz_raw = llm.invoke(prompt)

        clean_analiz  =json_temizle(analiz_raw)
         #try:
            # LLM çıktısını parse et ve mevcut soru objesini güncelle
        analiz = json.loads(clean_analiz)
        soru["METADATA"]["kavramlar"] = analiz.get("kavramlar")
        """except:

            print(f"Hata: {soru['SORU_ID']} analiz edilemedi.")"""

    return sorular_listesi

with open('/content/altin_sorular.json', 'r') as f:
    sorular = json.load(f)
güncel_sorular = kavramlari_etiketle(sorular)

İşleniyor: AOF_2021_YAZ_1
İşleniyor: AOF_2021_YAZ_2
İşleniyor: AOF_2021_YAZ_3
İşleniyor: AOF_2021_YAZ_4
İşleniyor: AOF_2021_YAZ_5
İşleniyor: AOF_2021_YAZ_6
İşleniyor: AOF_2021_YAZ_8
İşleniyor: AOF_2021_YAZ_9
İşleniyor: AOF_2021_YAZ_10
İşleniyor: AOF_2021_YAZ_11
İşleniyor: AOF_2021_YAZ_12
İşleniyor: AOF_2021_YAZ_13
İşleniyor: AOF_2021_YAZ_14
İşleniyor: AOF_2021_YAZ_15
İşleniyor: AOF_2021_YAZ_16
İşleniyor: AOF_2021_YAZ_17
İşleniyor: AOF_2021_YAZ_18
İşleniyor: AOF_2021_YAZ_19
İşleniyor: AOF_2021_YAZ_20


In [ ]:
güncel_sorular

[{'SORU_ID': 'AOF_2021_YAZ_1',
  'SORU': '1936 yılında yazdığı “İstihdam, Faiz ve Paranın Genel Teorisi” adlı eserinde, klasik iktisat anlayışına önemli eleştiriler getirerek ve liberal çizgiden ödünler vermiş olan iktisatçı kimdir?',
  'SIKLAR': {'A': 'Keynes',
   'B': 'Friedman',
   'C': 'Fisher',
   'D': 'Hayek',
   'E': 'Mill'},
  'CEVAP': 'A',
  'METADATA': {'unite_no': None,
   'kavramlar': ['İstihdam',
    'Faiz',
    'Paran',
    'Klasik İktisat',
    'Liberal Çizgi',
    'Keynes']}},
 {'SORU_ID': 'AOF_2021_YAZ_2',
  'SORU': 'Sanayi Devrimi’nin ortaya çıkardığı kötü çalışma koşullarına karşı işçileri ve emeği sermayeye karşı korumak ve bu yolla toplumdaki sınıf çatışmalarını önleyerek toplumun ve devletin varlığını sürdürmesini sağlamaya yönelik uygulamalara kavramsal açıdan ne ad verilir?',
  'SIKLAR': {'A': 'Sosyal yardım',
   'B': 'Geniş anlamda sosyal politika',
   'C': 'Sosyal sigorta',
   'D': 'Sosyal güvenlik',
   'E': 'Dar anlamda sosyal politika'},
  'CEVAP': 'E',
  'M

In [ ]:
# JSON olarak kaydet
with open("güncel_sorular.json", "w", encoding="utf-8") as f:
    json.dump(sonuc_listesi, f, ensure_ascii=False, indent=2)

In [ ]:
import re
import json

def altin_format_olustur(ham_metin, cevap_anahtari_str):
    # 1. Metni Temizle (Kurum mesajlarını ve sayfa numaralarını sil)
    temiz_metin = re.sub(r"Açıköğretim Sistemi ile ilgili.*?yasaktır\.", "", ham_metin, flags=re.DOTALL)
    temiz_metin = re.sub(r"Anadolu Üniversitesi.*?sayılır\.", "", temiz_metin, flags=re.DOTALL)
    # Sayfa numaraları ve sınav başlıklarını temizle
    temiz_metin = re.sub(r"\d{4} - \d{4} Öğretim Yılı|Yaz Okulu Sınavı|SOSYAL POLİTİKA A|2021 YAZ OKULU - A", "", temiz_metin)

    # 2. Cevap Anahtarını Hazırla
    cevap_listesi = cevap_anahtari_str.split()

    # 3. Soruları Ayıkla
    # Soru numarası (örn: 1.) ile başlayan blokları bulur
    soru_bloklari = re.split(r'\n(?=\d+\.)', temiz_metin)

    altin_liste = []
    soru_sayaci = 0

    for blok in soru_bloklari:
        # Soru numarası ve içeriği ayır
        match = re.match(r'(\d+)\.(.*)', blok.strip(), re.DOTALL)
        if match:
            soru_no_text = match.group(1)
            icerik = match.group(2)

            # Şıkları parçala (A, B, C, D, E)
            parts = re.split(r'\s([A-E])\)\s', icerik)

            if len(parts) > 1:
                soru_metni = parts[0].strip().replace('\n', ' ')
                siklar = {}
                for i in range(1, len(parts), 2):
                    if i+1 < len(parts):
                        sik_harfi = parts[i]
                        # Şık metnindeki gereksiz satır sonlarını temizle
                        sik_metni = parts[i+1].split('\n')[0].strip()
                        siklar[sik_harfi] = sik_metni

                # Soruya doğru cevabı ata
                soru_cevap = cevap_listesi[soru_sayaci] if soru_sayaci < len(cevap_listesi) else "Bilinmiyor"

                altin_liste.append({
                    "SORU_ID": f"AOF_2021_YAZ_{soru_no_text}",
                    "SORU": soru_metni,
                    "SIKLAR": siklar,
                    "CEVAP": soru_cevap,
                    "METADATA": {
                        "unite_no": None,
                        "kavramlar": []
                    }
                })
                soru_sayaci += 1

    return altin_liste

# --- UYGULAMA ---

ham_veri = """
1.Üretim faktörü olarak insanı ele alarak işgücü
piyasasında istihdam ve ücret konularını
inceleyen bilim dalı aşağıdakilerden
hangisidir?
A)
Sanayi ekonomisi
B)
Kalkınma ekonomisi
C)
Yeni ekonomi
D)
Çalışma ekonomisi
E)
Kamu ekonomisi

2.Liberalizme en ciddi ve tutarlı eleştirileri
getirerek bilimsel sosyalizmi geliştirilen
bilim adamı kimdir?
A)
Charles Fourier
B)
Karl Marks
C)
Robert Owen
D)
Saint Simon
E)

3.Jean-Jacques Rousseau
Aşağıdakilerden hangisi sosyal politikanın
kollektif kendi kendine yardım araçlarından
biri değildir?
A)
Sendikalar
B)
Bakanlıklar
C)
Dernekler
D)
Vakıflar
E)
Kooperatifler

4.İngiltere’de 1802 yılında, çalışma ilişkilerinin
düzenlenmesine ilişkin basit bir uygulama ile
başlayan müdahale anlayışı ilk olarak hangi
çalışan grubuna yönelik ortaya çıkmıştır?
A)
Madende çalışan kadınlar
B)
Sanayide çalışan erkekler
C)
Sanayide çalışan çocuklar
D)
Madende çalışan erkekler
E)
Sanayide çalışan kadınlar

5.İşçilere normal bir çalışma günü karşılığı
ödenen ve işçinin gıda, konut, giyim, sağlık,
ulaşım ve kültür gibi zorunlu ihtiyaçlarını
günün fiyatları üzerinden en az düzeyde
karşılamaya yetecek ücrete ne ad verilir?
A)
Cari ücret
B)
Reel ücret
C)
Ayni ücret
D)
Çıplak ücret
E)
Asgari ücret

6.Genellikle asgari bir yaşam düzeyini
sağlamak için gerekli olan gelir düzeyinin
yoksulluk sınırı olarak tanımlandığı yoksulluk
türü aşağıdakilerden hangisidir?
A)
Gelir yoksulluğu
B)
Toplumsal yoksulluk
C)
Mutlak yoksulluk
D)
Göreli yoksulluk
E)
Objektif yoksulluk

7.Kişilerin kendilerine sağlanan sosyal yardım
ve hizmetlerden herhangi bir bedel ya da
prim ödemeden yararlanmasına ne ad
verilir?
A)
Kayıtdışı istihdam
B)
Sosyal sigorta hakkı
C)
Sosyal destek
D)
Devlet destekli istihdam
E)
Primsiz sistem

8.Aşağıdakilerden hangisi pasif istihdam
politikalarından biridir?
A)
Doğrudan kamu istihdamı
Girişimciliğin desteklenmesine yönelik
programlar
B)
C)
İş paylaşımı
D)
Ücret ve istihdam sübvansiyonları
E)
Mesleki eğitim programları

9.Cinsel yönelimi, ten rengi veya dini inancı
nedeniyle bir kişiyi işe almamak aşağıdaki
ayrımcılık türlerinden hangisine örnek olarak
verilebilir?
A)
Sistematik ayrımcılık
B)
Doğrudan ayrımcılık
C)
Dolayısıyla ayrımcılık
D)
Dolaylı ayrımcılık
E)
Ters yönlü ayrımcılık

10.Aşağıdakilerden hangisi sosyal dışlanma
türlerinden biri değildir?
A)
Siyasi dışlanma
B)
Mutlak dışlanma
C)
Kültürel dışlanma
D)
Mekânsal dışlanma
E)
Ekonomik dışlanma

11.Dezavantajlı gruplara uzun süre uygulanan
olumlu ayrımcılık uygulamaları nedeniyle
toplumda genel olan grubun karşı karşıya
kaldığı ayrımcılık türüne ne ad verilir?
A)
Dolaylı ayrımcılık
B)
Dolayısıyla ayrımcılık
C)
Ters yönlü ayrımcılık
D)
Cinsel taciz
E)
Açık ayrımcılık

12.Aşağıdakilerden hangisi modern sosyal
güvenlik yöntemlerinden biridir?
A)
Bireysel tasarruflar
B)
Dini sosyal yardımlar
C)
Kurumsallaşmış sosyal yardımlar
D)
Sosyal sigorta
E)
Tanıma bilme faktörüne bağlı yardımlaşma

13.Sosyal sigortaların özellikleri ile ilgili
aşağıdaki ifadelerden hangisi yanlıştır?
A)
Devlet sosyal sigortaların garantörüdür.
Sosyal sigortalar devlet tarafından
kurulmasına rağmen, diğer kamu
kurumlarından farklı olarak yönetimine
sosyal taraflar da katılır.
B)
Sosyal sigortaların kapsamına girmek
zorunludur.
C)
Sosyal sigortalar, devlet tarafından kurulan
kamu sigorta programlarıdır.
D)
Sosyal sigortalarda ödenen primlerle
sağlanan haklar arasındaki bağlantı hastalık
sigortasında gelirin yeniden dağılımını
sağlayacak şekilde çok güçlüdür.
E)
Sosyal sigortalar, 19. yüzyılın son çeyreğinde
Almanya’da ilk kim tarafından kurulmuştur?
14.
A)
Carnot
B)
Owen
C)
Beveridge
D)
Churchill
E)
Bismarck

15.Kadınlar tarihin her döneminde çalışma
yaşamında sürekli olarak yer almıştır. Bununla
beraber ücretli işçi olarak çalışma yaşamına
girmeleri ilk kez ------- ile olmuştur.
Yukarıdaki cümlede boş bırakılan yeri
aşağıdakilerden hangisi doğru şekilde
tamamlar?
A)
Büyük Buhran
B)
I. Dünya Savaşının sona ermesi
C)
Loncaların kurulması
D)
Sanayi Devrimi
E)
Toprağın tarıma açılması


16.Türkiye’de engellilere çalıştırma zorunluluğu
ilk olarak hangi düzenlemede yer almıştır?
A)
854 sayılı Deniz İş Kanunu
B)
931 sayılı İş Kanunu
C)
1593 sayılı Umumi Hıfzıssıhha Kanunu
D)
1924 Anayasası
1974 tarihli İşçi Sağlığı ve İş Güvenliği
Tüzüğü
E)

17.------- göre; geniş anlamda küreselleşme,
ekonomik göstergelerle ilgili değişimin yanı sıra
küreselleşmenin kültürel, siyasal ve çevre ile ilgili
boyutlarını da kapsamaktadır.
Yukarıdaki cümlede boş bırakılan yeri
aşağıdakilerden hangisi doğru olarak
tamamlar?
A)
Uluslararası Çalışma Örgütü'ne
B)
Avrupa Birliği'ne
C)
Uluslararası Para Fonu'na
D)
Dünya Ticaret Örgütü'ne
E)
Birleşmiş Milletler'e

18.Küreselleşme sürecine daha çok taraflı ve
dengeli bir bakış açısıyla yaklaşan,
küreselleşmeyi nimetleri ve külfetleri;
fırsatları ve tehditleri ve nihayet olumlu ve
olumsuz sonuçları ile birlikte değerlendirmek
gerektiğini ileri süren yaklaşım
aşağıdakilerden hangisidir?
A)
Şüpheciler
B)
Reformistler
C)
Piyasacılar
D)
Dönüşümcüler
E)
Aşırı küreselleşmeciler

19.Yaşlıları tanımlamak için kullanılan kavramlar
bakımından "dördüncü çağ" kavramı kaç yaş
üstündekiler için kullanılmaktadır?
A)
80
B)
70
C)
65
D)
60
E)
55

20.“Herkes İçin Fırsatlar Yaratan Adil Bir
Küreselleşme Raporu” hangi kuruluş
tarafından hazırlanmıştır?
A)
Uluslararası Çalışma Örgütü
B)
Avrupa Sendikalar Konfederasyonu
C)
Dünya Bankası
D)
Dünya Sağlık Örgütü
E)
Birleşmiş Milletler
"""

cevap_anahtari="""D B B C E A E C B B C D E E D A C D A A"""

# Fonksiyonu çalıştır
sonuc_listesi = altin_format_olustur(ham_veri, cevap_anahtari)

# JSON olarak kaydet
with open("guncel_sorular_yaz_2.json", "w", encoding="utf-8") as f:
    json.dump(sonuc_listesi, f, ensure_ascii=False, indent=2)

print(f"İşlem Tamam: {len(sonuc_listesi)} soru cevaplarıyla birlikte 'altin_sorular.json' dosyasına kaydedildi.")

İşlem Tamam: 20 soru cevaplarıyla birlikte 'altin_sorular.json' dosyasına kaydedildi.


In [ ]:
import json
import re
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3")

def json_temizle(raw_text):
    """LLM yanıtındaki gereksiz metinleri temizleyip sadece İLK JSON bloğunu alır."""
    try:
        # İlk { ile ona karşılık gelen son } arasını bulmaya çalışır
        match = re.search(r'(\{.*?\})', raw_text, re.DOTALL)
        if match:
            return match.group(1)
        return raw_text
    except:
        return raw_text

def kavramlari_etiketle(sorular_listesi):
    SYSTEM_PROMPT = """
    Sen bir akademik içerik etiketleyicisin. SADECE JSON formatında yanıt ver.
    JSON dışında hiçbir açıklama veya ek metin yazma.

    ÖRNEK ÇIKTI:
    {
      "kavramlar": ["Kavram1", "Kavram2"]
    }
    """

    for soru in sorular_listesi:
        sid = soru.get('SORU_ID', 'Bilinmiyor')
        print(f"İşleniyor: {sid}")

        prompt = f"{SYSTEM_PROMPT}\n\nSoru: {soru['SORU']}\nŞıklar: {soru['SIKLAR']}"

        try:
            analiz_raw = llm.invoke(prompt)
            clean_analiz = json_temizle(analiz_raw)

            # JSON yüklemeyi dene
            analiz = json.loads(clean_analiz)

            # METADATA veya metadata anahtarını kontrol et
            meta_key = "METADATA" if "METADATA" in soru else "metadata"
            if meta_key not in soru:
                soru[meta_key] = {}

            soru[meta_key]["kavramlar"] = analiz.get("kavramlar", [])

        except json.JSONDecodeError:
            print(f"Hata: {sid} için geçersiz JSON formatı. Atlanıyor...")
        except Exception as e:
            print(f"Hata: {sid} işlenirken bir sorun oluştu: {str(e)}")

    return sorular_listesi

# Dosyayı yükle ve çalıştır
with open('/content/guncel_sorular_yaz_2.json', 'r') as f:
    sorular = json.load(f)

güncel_sorular = kavramlari_etiketle(sorular)

# Sonucu hemen kaydet (Data kaybını önlemek için)
with open('/content/altin_sorular_son.json', 'w', encoding='utf-8') as f:
    json.dump(güncel_sorular, f, ensure_ascii=False, indent=2)

print("İşlem bitti!")

İşleniyor: AOF_2021_YAZ_1
İşleniyor: AOF_2021_YAZ_2
İşleniyor: AOF_2021_YAZ_3
İşleniyor: AOF_2021_YAZ_4
İşleniyor: AOF_2021_YAZ_5
İşleniyor: AOF_2021_YAZ_6
İşleniyor: AOF_2021_YAZ_7
İşleniyor: AOF_2021_YAZ_8
İşleniyor: AOF_2021_YAZ_9
İşleniyor: AOF_2021_YAZ_10
İşleniyor: AOF_2021_YAZ_11
İşleniyor: AOF_2021_YAZ_12
İşleniyor: AOF_2021_YAZ_13
İşleniyor: AOF_2021_YAZ_14
İşleniyor: AOF_2021_YAZ_15
İşleniyor: AOF_2021_YAZ_16
İşleniyor: AOF_2021_YAZ_17
İşleniyor: AOF_2021_YAZ_18
İşleniyor: AOF_2021_YAZ_19
İşleniyor: AOF_2021_YAZ_20
İşlem bitti!


In [ ]:
import re
import json
import os
from langchain_ollama import OllamaLLM

# --- AYARLAR VE MODEL ---
llm = OllamaLLM(model="llama3")

# Google Drive veya yerel dizin yolları
input_klasoru = "/content/drive/MyDrive/cikmis_sorular"
cikti_klasoru = "/content/drive/MyDrive/islenmis_sorular/"

if not os.path.exists(cikti_klasoru):
    os.makedirs(cikti_klasoru)

# İşlenecek dosyalar ve cevap anahtarları
is_listesi = {
    "guz_ara_sinav_2022_cikmis_sorular": "A A D C D E B D C B B E E E A A B C D C",
    "guz_ara_sinav_2023_cikmis_sorular": "C E A D C A B B C C E A E A B B D D D E",
    "guz_ara_sinav_2024_cikmis_sorular": "E D C C B A E C D B C A B D E B A D A E",
    "yaz_okulu_2023_cikmis_sorular": "A A D C E C D B D C B D A E A B E E B C",
    "yaz_okulu_2024_cikmis_sorular": "B D D A B B C C C B E E C A E D A A D E",
    "yaz_okulu_2025_cikmis_sorular": "E D D E A B E B E C C C A A D D A B C B"
}

# --- YARDIMCI FONKSİYONLAR ---

def json_temizle(raw_text):
    """LLM'den gelen cevabı sadece JSON bloğu kalacak şekilde temizler."""
    try:
        # En dıştaki süslü parantezleri yakalar
        match = re.search(r'(\{.*?\})', raw_text, re.DOTALL)
        if match:
            return match.group(1)
        return raw_text
    except:
        return raw_text

def metni_jsona_cevir(ham_metin, cevap_anahtari_str, dosya_adi):
    """Ham metni parçalayıp temel JSON objesine dönüştürür."""
    cevaplar = cevap_anahtari_str.split()
    # Soru numarasıyla başlayan blokları ayırır (örn: 1., 2.)
    soru_bloklari = re.split(r'\n(?=\d+\.)', ham_metin.strip())

    altin_liste = []
    for i, blok in enumerate(soru_bloklari):
        match = re.match(r'(\d+)\.(.*)', blok.strip(), re.DOTALL)
        if match:
            icerik = match.group(2)
            # Şıkları ayırır (A), B) vb.)
            parts = re.split(r'\s([A-E])\)\s', icerik)
            if len(parts) > 1:
                soru_metni = parts[0].strip().replace('\n', ' ')
                siklar = {}
                for j in range(1, len(parts), 2):
                    if j+1 < len(parts):
                        sik_harfi = parts[j]
                        # Şık metnini temizle
                        sik_metni = parts[j+1].split('\n')[0].strip()
                        siklar[sik_harfi] = sik_metni

                altin_liste.append({
                    "SORU_ID": f"{dosya_adi}_{i+1}",
                    "SORU": soru_metni,
                    "SIKLAR": siklar,
                    "CEVAP": cevaplar[i] if i < len(cevaplar) else "Bilinmiyor",
                    "METADATA": {"unite_no": None, "kavramlar": []}
                })
    return altin_liste

# --- ANA DÖNGÜ ---

for dosya_id, cevap_anahtari in is_listesi.items():
    dosya_yolu = os.path.join(input_klasoru, f"{dosya_id}.txt")

    if not os.path.exists(dosya_yolu):
        print(f"{dosya_id}.txt bulunamadı, atlanıyor.")
        continue

    print(f"\n{dosya_id} İşleniyor...")

    with open(dosya_yolu, "r", encoding="utf-8") as f:
        ham_metin = f.read()

    # 1. Aşama: Yapısal Dönüşüm (Regex ile JSON formatına)
    soru_listesi = metni_jsona_cevir(ham_metin, cevap_anahtari, dosya_id)

    # 2. Aşama: LLM ile Kavram Etiketleme (Her soru için tek tek)
    print(f"LLM Kavram Analizi Yapıyor ({len(soru_listesi)} Soru)...")

    SYSTEM_PROMPT = """Sen bir akademik içerik etiketleyicisin.
    Verilen soruyu analiz et ve içindeki temel akademik kavramları çıkart.
    SADECE JSON formatında yanıt ver.
    Örnek: {"kavramlar": ["Sosyal Politika", "Sanayi Devrimi"]}"""

    for soru in soru_listesi:
        prompt = f"{SYSTEM_PROMPT}\n\nSoru: {soru['SORU']}\nŞıklar: {soru['SIKLAR']}"

        try:
            raw_response = llm.invoke(prompt)
            clean_json = json_temizle(raw_response)
            analiz = json.loads(clean_json)

            # Metadata içindeki kavramlar listesini güncelle
            soru["METADATA"]["kavramlar"] = analiz.get("kavramlar", [])
            print(f"   {soru['SORU_ID']} etiketlendi.")

        except Exception as e:
            print(f"   ❌soru['SORU_ID']} için kavramlar oluşturulamadı: {e}")
            continue

    # 3. Aşama: Dosyayı Kaydet
    kayit_adi = f"{dosya_id}_islenmis.json"
    cikti_yolu = os.path.join(cikti_klasoru, kayit_adi)

    with open(cikti_yolu, "w", encoding="utf-8") as f:
        json.dump(soru_listesi, f, ensure_ascii=False, indent=2)

    print(f"📁 {kayit_adi} başarıyla kaydedildi.")

print("\n✨ TÜM SÜREÇ TAMAMLANDI! Tüm dosyalar kavramlarıyla birlikte hazır.")


🚀 guz_ara_sinav_2022_cikmis_sorular İşleniyor...
🧠 LLM Kavram Analizi Yapıyor (20 Soru)...
   ✅ guz_ara_sinav_2022_cikmis_sorular_2 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_3 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_4 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_5 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_6 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_7 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_8 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_9 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_10 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_11 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_12 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_13 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_14 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_15 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_16 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorular_17 etiketlendi.
   ✅ guz_ara_sinav_2022_cikmis_sorul